## Replace dogs and cats with round and rectangle

'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input

Using TensorFlow backend.


In [2]:
# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 200, 400

In [11]:
! ls

additional_stats_mens.csv	    keras_demo_top_model.ipynb
additional_stats_womens.csv	    make_train_test_dir.ipynb
all_watch_info_case_shapes.csv	    prestige_time_pics_mens
all_watch_info.csv		    prestige_time_pics_womens
analyze_product_descriptions.ipynb  preview
band_data			    README.md
bottleneck_fc_model.h5		    scrape_prestigetime_pages.ipynb
bottleneck_features_train.npy	    shape_data
bottleneck_features_validation.npy  sloth.jpg
cnn_practice.ipynb		    synset_words.txt
dog.jpeg			    vgg16_weights_tf_dim_ordering_tf_kernels.h5
get_links_and_pics.ipynb	    watch_page_list_mens.csv
keras_demo.ipynb		    watch_page_list_womens.csv


In [3]:
train_data_dir = 'shape_data/train'
validation_data_dir = 'shape_data/test'
nb_train_samples = 6672
nb_validation_samples = 3288
epochs = 50
batch_size = 8


final_save_name = 'watch_shape_combo_attempt1_model.h5'

In [4]:
# build the VGG16 network
model = applications.VGG16(weights='imagenet',
                           include_top=False,
                          input_shape=(img_width, img_height, 3))
print('Model loaded.')

Model loaded.


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 400, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 400, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 400, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 200, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 200, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 200, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 100, 128)      0         
__________

In [6]:
input_tensor = Input(shape=(img_width, img_height, 3))
base_model = applications.VGG16(
    weights='imagenet',
    include_top= False,
    input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
top_model.load_weights('bottleneck_fc_model.h5')
model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

In [7]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

In [8]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [9]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 6678 images belonging to 2 classes.
Found 3295 images belonging to 2 classes.


In [15]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., steps_per_epoch=834, validation_steps=3288)`
  import sys


Epoch 1/50


ValueError: Error when checking input: expected input_2 to have shape (None, 200, 400, 3) but got array with shape (8, 400, 200, 3)

In [ ]:
model.save_weights(final_save_name)

In [ ]:
model.summary()